# ABT for defaulted agencies

In [2]:
pip install awswrangler

  Using cached awswrangler-3.9.0-py3-none-any.whl.metadata (17 kB)
Using cached awswrangler-3.9.0-py3-none-any.whl (381 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
!{sys.executable} -m pip install PyAthena

  Using cached pyathena-3.9.0-py3-none-any.whl.metadata (6.3 kB)
Using cached pyathena-3.9.0-py3-none-any.whl (75 kB)


In [4]:
import awswrangler as wr
import pandas as pd
import datetime as dt
from datetime import datetime
from pyathena import connect
import matplotlib.pyplot as plt
import seaborn as sns

## Get TX, amount and balance

### sumando viachek

In [5]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

df2 = pd.read_sql('''with viacheck as(
SELECT
    DATE_FORMAT(ct.CheckDate, '%Y-%m-%d') AS DATE,
    COUNT(DISTINCT ct.checkID) AS TX_check,
    SUM(ct.AMOUNT) AS TOTAL_AMOUNT_check,
    b.id_location
FROM
    AwsDataCatalog.viamericas.checkTable ct
LEFT JOIN
    AwsDataCatalog.viamericas.BRANCH a ON a.id_branch = ct.CheckBranch
LEFT JOIN
    AwsDataCatalog.viamericas.location b ON a.id_location = b.id_location
       
WHERE
   NOT (a.id_branch LIKE 'T%')
   AND a.id_flag_branch = 'A'
   AND DATE_FORMAT(ct.CheckDate, '%Y-%m') between '2023-01' and '2024-02'
   AND  NOT b.id_location = 'NULL'
GROUP BY
    DATE_FORMAT(ct.CheckDate, '%Y-%m-%d'),
    b.id_location
    --b.name_location;
)
SELECT
   CAST(RTRIM(b.ID_LOCATION) AS VARCHAR(60)) AS AGENCY,
   d.ID_GROUP_COMPANY AS ID_GROUP_COMPANY,
   a.ID_BRANCH as ID_BRANCH,
   CAST(DATE_FORMAT(a.DATE_RECEIVER, '%Y-%m-%d') AS DATE) AS DATE,
   SUM(CASE WHEN (A.ID_FLAG_RECEIVER = 'A' OR A.ID_FLAG_RECEIVER = 'C') THEN 0 ELSE 1 END) AS TX,
   SUM(CASE WHEN (A.ID_FLAG_RECEIVER = 'A' OR A.ID_FLAG_RECEIVER = 'C') THEN 0 ELSE A.NET_AMOUNT_RECEIVER END) as AMOUNT,
   c.BALANCE as BALANCE,
   coalesce(d.TX_check, 0) as TX_check,
   coalesce(d.TOTAL_AMOUNT_check, 0) as TOTAL_AMOUNT_check,
   c.dso_deposit_day as DSO_DEPO_DAY,
   CAST(DATE_FORMAT(d.installation_date, '%Y-%m-%d') AS DATE) as START_DATE
 FROM
   AwsDataCatalog.viamericas.RECEIVER a
   INNER JOIN AwsDataCatalog.viamericas.GROUP_BRANCH p ON p.ID_MAIN_BRANCH = CASE WHEN a.ID_MAIN_BRANCH_EXPIRED IS NULL THEN RTRIM(a.ID_MAIN_BRANCH_SENT) ELSE RTRIM(a.ID_MAIN_BRANCH_EXPIRED) END
   LEFT JOIN AwsDataCatalog.viamericas.BRANCH b ON a.ID_BRANCH = b.ID_BRANCH
   LEFT JOIN AwsDataCatalog.viamericas.history_balance c ON c.id_main_branch = b.id_main_branch and c.date_balance = cast(a.DATE_RECEIVER as DATE)
   LEFT JOIN viacheck d ON d.id_location = b.ID_LOCATION and cast(d.DATE as DATE) = cast(a.DATE_RECEIVER as DATE)
   left join AwsDataCatalog.viamericas.locations d on b.ID_LOCATION = d.id
   
 WHERE
   NOT (A.ID_MAIN_BRANCH_SENT LIKE 'M%')
   AND NOT (A.ID_BRANCH LIKE 'T%')
   AND SUBSTRING(b.id_branch, 1, 1) IN (SELECT SUBSTRING(PREFIX, 1, 1) FROM AwsDataCatalog.viamericas.BRANCH_PREFIX P WHERE TRIM(B.ID_COUNTRY) = TRIM(P.ID_COUNTRY))
   AND A.DATE_RECEIVER >= TIMESTAMP '2023-01-01 00:00:00.000'
   AND A.DATE_RECEIVER < TIMESTAMP '2024-02-04 00:00:00.000'
   AND b.ID_LOCATION IS NOT NULL
   AND b.ID_LOCATION NOT IN ('MD0010', 'MD0952', 'AK0003', 'CA3897', 'NY1130', 'MD0696', 'FL1933', 'AK0004', 'CA4046', 'NY1221', 'MD0623', 'MD1003', 'MD1018', 'AK0008', 'CA4291', 'NY1346', 'CA4350', 'NY1381','OK0236','FL2287','CA4391','AK0009', 'NY1397','CA4392','AK0010','FL2288','NY1399','AK0012','CA4396','NY1402','FL2289','AK0013', 'CA4418','NY1410','FL2301','AK0014', 'CA4428', 'NY1413')
   AND a.NET_AMOUNT_RECEIVER <> 0
 GROUP BY
   RTRIM(b.ID_LOCATION),
   d.ID_GROUP_COMPANY,
   a.ID_BRANCH,
   c.BALANCE, 
   d.TX_check,
   d.TOTAL_AMOUNT_check,
   c.dso_deposit_day,
   d.installation_date,
   CAST(DATE_FORMAT(a.DATE_RECEIVER, '%Y-%m-%d') AS DATE); ''', conn)

/tmp/ipykernel_395/1018844275.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql('''with viacheck as(


In [6]:
#df = pd.read_csv('2023_trx_balance_history3.csv')

#df2= pd.read_csv('2023_trx_balance_check_history3.csv')
mo = pd.read_csv('MO.csv')

In [7]:
df2.columns

Index(['AGENCY', 'ID_GROUP_COMPANY', 'ID_BRANCH', 'DATE', 'TX', 'AMOUNT',
       'BALANCE', 'TX_check', 'TOTAL_AMOUNT_check', 'DSO_DEPO_DAY',
       'START_DATE'],
      dtype='object')

In [8]:
mo.columns

Index(['DATE', 'unique_senders', 'TX', 'TX_CANCELLED', 'TOTAL_AMOUNT', 'GP',
       'state', 'id_chain', 'name_chain', 'id_location', 'name_location',
       'id_branch', 'name_branch', 'kpr', 'LATITUD', 'LONGITUD', 'ZIP',
       'id_agent', 'TYPE_PRODUCT'],
      dtype='object')

In [9]:
#filtrar variables relevantes
lista = ['DATE','id_location', 'TX','TOTAL_AMOUNT']
mo2 = mo[lista]

In [10]:
#renombrar variables para que coincidan con el formato del resto
mo2.rename(columns={'DATE': 'DATE', 'id_location': 'AGENCY', 'TX':'TX_MO','TOTAL_AMOUNT':'TOTAL_AMOUNT_MO'}, inplace = True)

/tmp/ipykernel_395/1387433609.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo2.rename(columns={'DATE': 'DATE', 'id_location': 'AGENCY', 'TX':'TX_MO','TOTAL_AMOUNT':'TOTAL_AMOUNT_MO'}, inplace = True)


In [11]:
# getting all dates in date format
mo2['DATE'] = pd.to_datetime(mo2['DATE'])
df2['DATE'] = pd.to_datetime(df2['DATE'])

/tmp/ipykernel_395/3607589272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo2['DATE'] = pd.to_datetime(mo2['DATE'])


In [12]:
mo2['DATE'] = mo2['DATE'].dt.strftime('%Y-%m-%d')
df2['DATE'] = df2['DATE'].dt.strftime('%Y-%m-%d')

/tmp/ipykernel_395/18446659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo2['DATE'] = mo2['DATE'].dt.strftime('%Y-%m-%d')


In [13]:
merged_df2 = pd.merge(df2, mo2, on=['DATE', 'AGENCY'], how='outer') #1661391 entries

In [14]:
merged_df2.drop_duplicates(subset=['DATE', 'AGENCY'], inplace = True) #1652242 entries

In [15]:
merged_df2.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 1652242 entries, 0 to 1661390
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   AGENCY              1652242 non-null  object 
 1   ID_GROUP_COMPANY    1650084 non-null  object 
 2   ID_BRANCH           1650084 non-null  object 
 3   DATE                1652242 non-null  object 
 4   TX                  1650084 non-null  float64
 5   AMOUNT              1650084 non-null  float64
 6   BALANCE             1649789 non-null  float64
 7   TX_check            1650084 non-null  float64
 8   TOTAL_AMOUNT_check  1650084 non-null  float64
 9   DSO_DEPO_DAY        1500215 non-null  float64
 10  START_DATE          1649459 non-null  object 
 11  TX_MO               138578 non-null   float64
 12  TOTAL_AMOUNT_MO     138578 non-null   float64
dtypes: float64(8), object(5)
memory usage: 176.5+ MB


In [16]:
default = pd.read_csv('Datos Juan.csv')#no podemos correr la querry en athena nos tienen que pasar el .csv de sql server

In [17]:
default['ID_BRANCH'].unique()#vienen del .csv con espaciosssss

array(['A00152    ', 'A00216    ', 'A00219    ', ..., 'A26036    ',
       'A26081    ', 'AB0110    '], dtype=object)

In [18]:
#quitando los espacios
default['ID_BRANCH'] = default['ID_BRANCH'].str.replace(' ','') 

In [19]:
#tengo defaults desde 2022-01, necesito sólo a partir de 2023-01-01
default = default[default['INACTIVATION_DATE'] > '2023-01-01']
default['INACTIVATION_DATE'].min() #check

'2023-01-01 09:31:38.730'

In [20]:
df3 = default[default['INACTIVE_DAYS'] > 6] #me quedo con 7 días inactivos en adelante

In [21]:
df3['DEFAULT'] = 1

/tmp/ipykernel_395/3250838113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['DEFAULT'] = 1


In [22]:
df3['DATE'] = pd.to_datetime(df3['INACTIVATION_DATE'])

/tmp/ipykernel_395/1161130008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['DATE'] = pd.to_datetime(df3['INACTIVATION_DATE'])


In [23]:
lista =['DATE', 'ID_BRANCH', 'DEFAULT']
df3 = df3[lista]

In [24]:
df3.shape

(2945, 3)

In [25]:
df3.head()

,DATE,ID_BRANCH,DEFAULT
29,2023-06-26 13:38:50.233,A00216,1
55,2023-08-21 12:19:06.193,A00219,1
67,2024-01-17 17:42:57.060,A00219,1
68,2024-01-31 16:23:23.357,A00219,1
81,2023-04-19 17:47:01.103,A00337,1


In [26]:
# hay que darle el formato '%Y-%m-%d' a la fechas de default!
df3['DATE'] = df3['DATE'].dt.strftime('%Y-%m-%d')

In [27]:
merged_df2['DATE'] = pd.to_datetime(merged_df2['DATE'])

In [28]:
merged_df2['DATE'] = merged_df2['DATE'].dt.strftime('%Y-%m-%d')

In [29]:
merged_df3 = pd.merge(merged_df2, df3, on=['DATE', 'ID_BRANCH'], how='outer')


In [30]:
merged_df3.drop_duplicates(subset=['DATE', 'ID_BRANCH'], inplace = True)

# agregar todos los inactive days

In [31]:
default['DATE'] = pd.to_datetime(default['INACTIVATION_DATE'])

In [32]:
default['DATE'] = default['DATE'].dt.strftime('%Y-%m-%d')

In [33]:
lista =['DATE', 'ID_BRANCH', 'INACTIVE_DAYS']
df5 = default[lista]

In [34]:
df5.shape

(22061, 3)

In [35]:
merged_df3 = pd.merge(merged_df3, df5, on=['DATE', 'ID_BRANCH'], how='outer')


In [36]:
merged_df3.drop_duplicates(subset=['DATE', 'ID_BRANCH'], inplace = True)

In [37]:
merged_df3.head()

,AGENCY,ID_GROUP_COMPANY,ID_BRANCH,DATE,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,DSO_DEPO_DAY,START_DATE,TX_MO,TOTAL_AMOUNT_MO,DEFAULT,INACTIVE_DAYS
0,WA0064,USA009517,A00270,2023-01-01,8.0,3769.0,14204.0018,0.0,0.00,0.0,2013-07-10,NaN,NaN,NaN,NaN
1,FL0096,USA000235,A00403,2023-01-01,1.0,500.0,20649.8329,20.0,13031.74,9.0,2005-02-14,NaN,NaN,NaN,NaN
2,NJ0080,USA000414,A00584,2023-01-01,12.0,7430.0,34849.6197,7.0,1939.68,2.0,2005-02-08,NaN,NaN,NaN,NaN
3,MD0145,USA000509,A00684,2023-01-01,23.0,12679.0,34991.7530,0.0,0.00,2.0,2005-12-22,NaN,NaN,NaN,NaN
4,MD0430,USA000609,A00788,2023-01-01,1.0,370.0,32049.7373,0.0,0.00,26.0,2008-10-31,NaN,NaN,NaN,NaN


In [38]:
merged_df3['DSO_DEPO_DAY'].unique()

array([  0.,   9.,   2.,  26.,   4.,   6.,   3.,  nan,   7.,  27.,   5.,
        10.,  24.,  12.,  17.,   1.,  28.,  11.,  22.,   8.,  13.,  14.,
        25.,  18.,  15.,  19.,  16.,  20.,  53.,  29.,  54.,  42.,  32.,
        61.,  33.,  35.,  23.,  21.,  78.,  79.,  30.,  34.,  80.,  31.,
        36.,  37.,  38.,  39.,  43.,  44.,  45.,  50.,  51.,  41.,  59.,
        58.,  75.,  52., 113.,  46.,  56.,  57., 131.,  64., 110., 162.,
        95.,  47., 187., 188., 189., 194., 127.,  40.,  66., 267., 200.,
        62.,  71.,  72.,  73.,  74.,  76.,  77.,  81.,  85.,  86., 316.,
        88., 150., 151., 152., 323.,  83.,  96., 330.])

In [39]:
merged_df3.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 1661114 entries, 0 to 1661363
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   AGENCY              1650473 non-null  object 
 1   ID_GROUP_COMPANY    1650084 non-null  object 
 2   ID_BRANCH           1660725 non-null  object 
 3   DATE                1661114 non-null  object 
 4   TX                  1650084 non-null  float64
 5   AMOUNT              1650084 non-null  float64
 6   BALANCE             1649789 non-null  float64
 7   TX_check            1650084 non-null  float64
 8   TOTAL_AMOUNT_check  1650084 non-null  float64
 9   DSO_DEPO_DAY        1500215 non-null  float64
 10  START_DATE          1649459 non-null  object 
 11  TX_MO               136809 non-null   float64
 12  TOTAL_AMOUNT_MO     136809 non-null   float64
 13  DEFAULT             2945 non-null     float64
 14  INACTIVE_DAYS       21811 non-null    float64
dtypes: float64(10), obje

In [40]:
merged_df3.to_csv('2023_trx_balance_check_default_history5.csv', index = False)

In [41]:
# Hay que unir de audit_credit (a nivel id_main_branch) new_credit_wknd(viernes, sab. dom y lun) y new_credit)martes,mi y jueves) para restar al balance y sacar el % del credito utilizado

## Reabrir y continuar

In [42]:
df = pd.read_csv('2023_trx_balance_check_default_history5.csv')

In [43]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1661114 entries, 0 to 1661113
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   AGENCY              1650473 non-null  object 
 1   ID_GROUP_COMPANY    1650084 non-null  object 
 2   ID_BRANCH           1660725 non-null  object 
 3   DATE                1661114 non-null  object 
 4   TX                  1650084 non-null  float64
 5   AMOUNT              1650084 non-null  float64
 6   BALANCE             1649789 non-null  float64
 7   TX_check            1650084 non-null  float64
 8   TOTAL_AMOUNT_check  1650084 non-null  float64
 9   DSO_DEPO_DAY        1500215 non-null  float64
 10  START_DATE          1649459 non-null  object 
 11  TX_MO               136809 non-null   float64
 12  TOTAL_AMOUNT_MO     136809 non-null   float64
 13  DEFAULT             2945 non-null     float64
 14  INACTIVE_DAYS       21811 non-null    float64
dtypes: float64(10),

Para mejorar expansión de fechas, primero lo aplico sólo para date y id_branch, luego mergeo left varibles dinámicas por date e id_branch y variables estáticas sólo por id_branch

In [44]:
#creo df básico para rellenar con fechas
lista =['DATE', 'ID_BRANCH']
stump = df[lista]

In [45]:
stump.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1661114 entries, 0 to 1661113
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   DATE       1661114 non-null  object
 1   ID_BRANCH  1660725 non-null  object
dtypes: object(2)
memory usage: 25.3+ MB


In [46]:
# versión 3.0 para expandir lo mínimo
def fill_missing_dates(df):
    # Create an empty DataFrame with the specified date range
    date_range = pd.date_range(start=df['DATE'].min(), end=df['DATE'].max())
    df_fill = pd.DataFrame({'DATE': date_range})
    df_fill['DATE'] = pd.to_datetime(df_fill['DATE']).dt.date

    # Sort the original DataFrame by 'AGENCY' and 'DATE'
    df = df.sort_values(by=['ID_BRANCH', 'DATE'])

    # Create an empty DataFrame to hold the result
    result_df = pd.DataFrame()

    # Loop through each 'AGENCY'
    for agency in df['ID_BRANCH'].unique():
        df_aux = df[df['ID_BRANCH'] == agency]
        # Combining df_aux (AGENCY) with df_fill, keeping values from df_aux and filling dates, value1, value2 and value3
        merged_df = df_aux.set_index('DATE').combine_first(df_fill.set_index('DATE')).reset_index()
        
    # Fill missing values in 'other_column1', 'other_column2', ... columns
        columns_to_fill = ['ID_BRANCH']
        merged_df[columns_to_fill] = merged_df[columns_to_fill].ffill().bfill()

        result_df = pd.concat([result_df, merged_df], ignore_index=True)

    return result_df


In [47]:
result_df = fill_missing_dates(stump) # ups era con el stump, funcionara?

In [48]:
result_df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4505196 entries, 0 to 4505195
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   DATE       4505196 non-null  object
 1   ID_BRANCH  4504797 non-null  object
dtypes: object(2)
memory usage: 68.7+ MB


In [49]:
result_df.head()

,DATE,ID_BRANCH
0,2023-01-02,A00002
1,2023-01-04,A00002
2,2023-01-05,A00002
3,2023-01-09,A00002
4,2023-01-10,A00002


In [50]:
result_df.to_csv('all_dates_branch.csv', index = False)

# Reanudando desde aquí

In [51]:
result_df = pd.read_csv('all_dates_branch.csv')

In [52]:
df = pd.read_csv('2023_trx_balance_check_default_history5.csv')

In [53]:
lista =['DATE', 'ID_BRANCH', 'TX', 'AMOUNT','BALANCE','TX_check','TOTAL_AMOUNT_check','TX_MO','TOTAL_AMOUNT_MO','DSO_DEPO_DAY', 'INACTIVE_DAYS','DEFAULT']
dinamic = df[lista]

In [54]:
merged_df1 = pd.merge(result_df, dinamic, on=['DATE', 'ID_BRANCH'], how='left')

In [55]:
lista =['ID_BRANCH', 'AGENCY', 'ID_GROUP_COMPANY','START_DATE']
static = df[lista]

In [56]:
static2 = static.drop_duplicates(subset=['ID_BRANCH'])

In [57]:
static2.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 7129 entries, 0 to 1661111
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID_BRANCH         7128 non-null   object
 1   AGENCY            6938 non-null   object
 2   ID_GROUP_COMPANY  6937 non-null   object
 3   START_DATE        6928 non-null   object
dtypes: object(4)
memory usage: 278.5+ KB


In [58]:
merged_df2 = pd.merge(merged_df1, static2, on=['ID_BRANCH'], how='left')

In [59]:
merged_df2.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4505196 entries, 0 to 4505195
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   DATE                4505196 non-null  object 
 1   ID_BRANCH           4504797 non-null  object 
 2   TX                  3300168 non-null  float64
 3   AMOUNT              3300168 non-null  float64
 4   BALANCE             3299578 non-null  float64
 5   TX_check            3300168 non-null  float64
 6   TOTAL_AMOUNT_check  3300168 non-null  float64
 7   TX_MO               273229 non-null   float64
 8   TOTAL_AMOUNT_MO     273229 non-null   float64
 9   DSO_DEPO_DAY        3000430 non-null  float64
 10  INACTIVE_DAYS       43622 non-null    float64
 11  DEFAULT             5890 non-null     float64
 12  AGENCY              4419869 non-null  object 
 13  ID_GROUP_COMPANY    4419470 non-null  object 
 14  START_DATE          4415251 non-null  object 
dtypes: float64(10),

In [60]:
merged_df2.to_csv('default_ABT0.csv', index = False)

# Crear target, default 9 días antes y extraer de fecha día de la semana

In [5]:
df = pd.read_csv('default_ABT0.csv')

In [6]:
df['DEFAULT'] = df['DEFAULT'].fillna(0)

In [7]:
# add day of week
df['DATE'] = pd.to_datetime(df['DATE'], format='mixed')
df['day_of_week'] = df['DATE'].dt.day_name()

In [8]:
# probar el duplicated para quedarse con los datos buenos y ahí probar generar la target!
df['filtro'] = df.duplicated(subset=['DATE', 'ID_BRANCH'], keep='first')


In [9]:
d8 = df[df['filtro'] == False]
d8.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 2844471 entries, 0 to 4505195
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   DATE                2844471 non-null  datetime64[ns]
 1   ID_BRANCH           2844072 non-null  object        
 2   TX                  1650084 non-null  float64       
 3   AMOUNT              1650084 non-null  float64       
 4   BALANCE             1649789 non-null  float64       
 5   TX_check            1650084 non-null  float64       
 6   TOTAL_AMOUNT_check  1650084 non-null  float64       
 7   TX_MO               136809 non-null   float64       
 8   TOTAL_AMOUNT_MO     136809 non-null   float64       
 9   DSO_DEPO_DAY        1500215 non-null  float64       
 10  INACTIVE_DAYS       21811 non-null    float64       
 11  DEFAULT             2844471 non-null  float64       
 12  AGENCY              2768262 non-null  object        
 13  ID_GROUP_COMPANY 

In [10]:
d8['DATE'] = d8['DATE'].dt.strftime('%Y-%m-%d')
d8['DATE1'] = d8['DATE'].astype(str)
d8['DATE_ID_BRANCH'] = d8['DATE1'] + '_' + d8['ID_BRANCH']

/tmp/ipykernel_21/123235449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d8['DATE'] = d8['DATE'].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_21/123235449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d8['DATE1'] = d8['DATE'].astype(str)
/tmp/ipykernel_21/123235449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [11]:
d8['filtro2'] = d8.duplicated(subset=['DATE_ID_BRANCH'], keep='first')

/tmp/ipykernel_21/2001156973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d8['filtro2'] = d8.duplicated(subset=['DATE_ID_BRANCH'], keep='first')


In [12]:
d9 = d8[d8['filtro2'] == False]
d9.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 2844073 entries, 0 to 4504797
Data columns (total 20 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   DATE                2844073 non-null  object 
 1   ID_BRANCH           2844072 non-null  object 
 2   TX                  1650084 non-null  float64
 3   AMOUNT              1650084 non-null  float64
 4   BALANCE             1649789 non-null  float64
 5   TX_check            1650084 non-null  float64
 6   TOTAL_AMOUNT_check  1650084 non-null  float64
 7   TX_MO               136421 non-null   float64
 8   TOTAL_AMOUNT_MO     136421 non-null   float64
 9   DSO_DEPO_DAY        1500215 non-null  float64
 10  INACTIVE_DAYS       21811 non-null    float64
 11  DEFAULT             2844073 non-null  float64
 12  AGENCY              2767864 non-null  object 
 13  ID_GROUP_COMPANY    2767863 non-null  object 
 14  START_DATE          2764272 non-null  object 
 15  day_of_week         

In [13]:
# otra forma de hacer la target, y es la posta!
d9=d9.sort_values(by=['ID_BRANCH', 'DATE'])
d9['TARGET'] =d9.groupby('ID_BRANCH')['DEFAULT'].shift(-9)

In [14]:
# borrar columnas no deseadas
lista = ['filtro', 'DATE1', 'DATE_ID_BRANCH', 'filtro2']
d9 = d9.drop(lista, axis=1)

In [15]:
# guradar
d9.to_csv('default_ABT1.csv', index = False)

In [23]:
# Target default mondays inactivation wednesday next week, lag 9 days, at slicing could serve tue and wed also
df['TARGET'] = df.set_index('DATE').groupby('ID_BRANCH', sort=False)['DEFAULT'].shift(-9).to_numpy()
# You can fill this NaN with 0 or another appropriate value
df['TARGET'] = df['TARGET'].fillna(0)

In [8]:
df['DEFAULT'].sum() # vs default original 5226 registros, sin fillna de default daba 3155, con fillna sigue dando 3155, con fechas formato homogeneo 8347
# 8941/10452 con id_branch sin espacios gano alguna covertura

5890.0

In [24]:
df.head()

,DATE,ID_BRANCH,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,TX_MO,TOTAL_AMOUNT_MO,DSO_DEPO_DAY,INACTIVE_DAYS,DEFAULT,AGENCY,ID_GROUP_COMPANY,START_DATE,day_of_week,filtro,TARGET
0,2023-01-02,A00002,0.0,0.0,13853.8257,0.0,0.0,NaN,NaN,0.0,NaN,0.0,MD0002,USA000002,2018-10-02,Monday,False,0.0
1,2023-01-04,A00002,1.0,340.0,13853.8257,0.0,0.0,NaN,NaN,0.0,NaN,0.0,MD0002,USA000002,2018-10-02,Wednesday,False,0.0
2,2023-01-05,A00002,1.0,25.0,14202.1057,0.0,0.0,NaN,NaN,0.0,NaN,0.0,MD0002,USA000002,2018-10-02,Thursday,False,0.0
3,2023-01-09,A00002,6.0,6.0,14232.1057,0.0,0.0,NaN,NaN,0.0,NaN,0.0,MD0002,USA000002,2018-10-02,Monday,False,0.0
4,2023-01-10,A00002,1.0,1.0,14268.1057,0.0,0.0,NaN,NaN,0.0,NaN,0.0,MD0002,USA000002,2018-10-02,Tuesday,False,0.0


In [10]:
 df['TARGET'].sum()

5410.0

In [11]:
5410/5890

0.9185059422750425

In [22]:
8941/10452

0.8554343666283964

In [20]:
8347/10452 #con las fechas bien gané de 0.6037 a 0.7986 de cobertura target/default

0.798603138155377

In [14]:
3155/5226

0.6037122081898201

In [21]:
d2 = df[df['DATE'] < '2022-01-12']

In [22]:
d2['DEFAULT'].sum()

89.0

In [23]:
5226-89 # este sería el numero total de target por los defaults de los primeros días que serian sacados del date_range original

5137

In [29]:
# casteando id_branch como str? NOPPP
df['ID_BRANCH'] =df['ID_BRANCH'].astype(str)

In [51]:
date_only = d4['DATE'].dt.date
num_days = len(date_only.unique())
num_days

761

In [9]:
# probar el duplicated para quedarse con los datos buenos y ahí probar generar la target!
df['filtro'] = df.duplicated(subset=['DATE', 'ID_BRANCH'], keep='first')


In [10]:
d7 = df[df['filtro'] == True]

In [11]:
d7.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 1660725 entries, 266 to 4504759
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   DATE                1660725 non-null  datetime64[ns]
 1   ID_BRANCH           1660725 non-null  object        
 2   TX                  1650084 non-null  float64       
 3   AMOUNT              1650084 non-null  float64       
 4   BALANCE             1649789 non-null  float64       
 5   TX_check            1650084 non-null  float64       
 6   TOTAL_AMOUNT_check  1650084 non-null  float64       
 7   TX_MO               136420 non-null   float64       
 8   TOTAL_AMOUNT_MO     136420 non-null   float64       
 9   DSO_DEPO_DAY        1500215 non-null  float64       
 10  INACTIVE_DAYS       21811 non-null    float64       
 11  DEFAULT             1660725 non-null  float64       
 12  AGENCY              1651607 non-null  object        
 13  ID_GROUP_COMPAN

In [12]:
d8 = df[df['filtro'] == False]
d8.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 2844471 entries, 0 to 4505195
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   DATE                2844471 non-null  datetime64[ns]
 1   ID_BRANCH           2844072 non-null  object        
 2   TX                  1650084 non-null  float64       
 3   AMOUNT              1650084 non-null  float64       
 4   BALANCE             1649789 non-null  float64       
 5   TX_check            1650084 non-null  float64       
 6   TOTAL_AMOUNT_check  1650084 non-null  float64       
 7   TX_MO               136809 non-null   float64       
 8   TOTAL_AMOUNT_MO     136809 non-null   float64       
 9   DSO_DEPO_DAY        1500215 non-null  float64       
 10  INACTIVE_DAYS       21811 non-null    float64       
 11  DEFAULT             2844471 non-null  float64       
 12  AGENCY              2768262 non-null  object        
 13  ID_GROUP_COMPANY 

In [13]:
86640/1650084 #pierdo un 5% de las días con tx.... filtrar y comparar, No se pierden están duplicados con datos!!!!

0.05250641785509101

In [14]:
1645312/6760398 #24% de repetidos

0.243375020228099

In [15]:
# sigue habiendo duplicados en d8? si formateo la fecha a sólo año-mes-día ayuda?
d8['DATE'] = d8['DATE'].dt.strftime('%Y-%m-%d')
d8['DATE1'] = d8['DATE'].astype(str)
d8['DATE_ID_BRANCH'] = d8['DATE1'] + '_' + d8['ID_BRANCH']

/tmp/ipykernel_18/1835377372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d8['DATE'] = d8['DATE'].dt.strftime('%Y-%m-%d')
/tmp/ipykernel_18/1835377372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d8['DATE1'] = d8['DATE'].astype(str)
/tmp/ipykernel_18/1835377372.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [16]:
d8['filtro2'] = d8.duplicated(subset=['DATE_ID_BRANCH'], keep='first')

/tmp/ipykernel_18/2001156973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d8['filtro2'] = d8.duplicated(subset=['DATE_ID_BRANCH'], keep='first')


In [17]:
d9 = d8[d8['filtro2'] == False]
d9.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 2844073 entries, 0 to 4504797
Data columns (total 20 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   DATE                2844073 non-null  object 
 1   ID_BRANCH           2844072 non-null  object 
 2   TX                  1650084 non-null  float64
 3   AMOUNT              1650084 non-null  float64
 4   BALANCE             1649789 non-null  float64
 5   TX_check            1650084 non-null  float64
 6   TOTAL_AMOUNT_check  1650084 non-null  float64
 7   TX_MO               136421 non-null   float64
 8   TOTAL_AMOUNT_MO     136421 non-null   float64
 9   DSO_DEPO_DAY        1500215 non-null  float64
 10  INACTIVE_DAYS       21811 non-null    float64
 11  DEFAULT             2844073 non-null  float64
 12  AGENCY              2767864 non-null  object 
 13  ID_GROUP_COMPANY    2767863 non-null  object 
 14  START_DATE          2764272 non-null  object 
 15  day_of_week         

In [18]:
d8['DEFAULT'].sum()

2945.0

d8['TARGET'].sum()#target calculado con los duplicados iguala y supera cantidad de defaults sin duplicados pero parece que le pega al menos a todos los casos

In [36]:
#código alterno para llenar todas las fechas para cada id_branch
df_full = d8.set_index('DATE').groupby('ID_BRANCH').apply( lambda x: x.reindex(pd.date_range(start=x.index.min(), end=x.index.max(), freq='D')) ).reset_index(level=0, drop=True).reset_index()
#kinda borró todos los datos....

/tmp/ipykernel_18/1185361185.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_full = d8.set_index('DATE').groupby('ID_BRANCH').apply( lambda x: x.reindex(pd.date_range(start=x.index.min(), end=x.index.max(), freq='D')) ).reset_index(level=0, drop=True).reset_index()


In [40]:
df_full.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2844072 entries, 0 to 2844071
Data columns (total 21 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   index               2844072 non-null  datetime64[ns]
 1   ID_BRANCH           0 non-null        object        
 2   TX                  0 non-null        float64       
 3   AMOUNT              0 non-null        float64       
 4   BALANCE             0 non-null        float64       
 5   TX_check            0 non-null        float64       
 6   TOTAL_AMOUNT_check  0 non-null        float64       
 7   TX_MO               0 non-null        float64       
 8   TOTAL_AMOUNT_MO     0 non-null        float64       
 9   DSO_DEPO_DAY        0 non-null        float64       
 10  INACTIVE_DAYS       0 non-null        float64       
 11  DEFAULT             0 non-null        float64       
 12  AGENCY              0 non-null        object        
 13  ID_GROUP_COM

In [41]:
# otra forma de hacer la target, y es la posta!
d8=d8.sort_values(by=['ID_BRANCH', 'DATE'])
d8['TARGET3'] =d8.groupby('ID_BRANCH')['DEFAULT'].shift(-9)

In [21]:
d8['TARGET2'].sum() #fail total absoluto

2483.0

In [23]:
2483/2945

0.8431239388794567

In [42]:
d8['TARGET3'].sum()

2873.0

In [43]:
2873/2945

0.9755517826825127

In [30]:
grouped_df = d8.groupby('ID_BRANCH').agg({ 'DEFAULT': 'sum', 'TARGET2': 'sum' }).reset_index() 
# Filtramos los casos donde las sumas sean distintas 
result_df = grouped_df[grouped_df['DEFAULT'] != grouped_df['TARGET2']]

In [31]:
result_df.head()

,ID_BRANCH,DEFAULT,TARGET2
14,A00337,1.0,0.0
50,A01300,1.0,0.0
77,A02091,3.0,0.0
89,A02420,1.0,0.0
156,A04126,1.0,0.0


In [44]:
result_df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 368 entries, 14 to 7021
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID_BRANCH  368 non-null    object 
 1   DEFAULT    368 non-null    float64
 2   TARGET2    368 non-null    float64
dtypes: float64(2), object(1)
memory usage: 11.5+ KB


In [45]:
grouped_df = d8.groupby('ID_BRANCH').agg({ 'DEFAULT': 'sum', 'TARGET3': 'sum' }).reset_index() 
# Filtramos los casos donde las sumas sean distintas 
result_df = grouped_df[grouped_df['DEFAULT'] != grouped_df['TARGET3']]

In [46]:
result_df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Index: 72 entries, 278 to 4761
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID_BRANCH  72 non-null     object 
 1   DEFAULT    72 non-null     float64
 2   TARGET3    72 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.2+ KB


In [47]:
result_df.head() #parece que sólo se perdieron los de primeros 9 días enero!

,ID_BRANCH,DEFAULT,TARGET3
278,A06713,4.0,3.0
476,A10458,2.0,1.0
477,A10495,8.0,7.0
892,A14583,1.0,0.0
938,A14848,7.0,6.0


In [61]:
# filtro por default =1 pero al filtrar por esos id_branches la tabla resultante no tiene default....Ahora q saque espacio de id_branch si
d11 = d8[d8['ID_BRANCH'] == 'A10458']

In [62]:
d11['DEFAULT'].sum()

2.0

In [63]:
d11[d11['DEFAULT'] == 1]

,DATE,ID_BRANCH,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,TX_MO,TOTAL_AMOUNT_MO,DSO_DEPO_DAY,...,AGENCY,ID_GROUP_COMPANY,START_DATE,day_of_week,filtro,DATE1,DATE_ID_BRANCH,filtro2,TARGET2,TARGET3
337570,2023-01-03,A10458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Tuesday,False,2023-01-03,2023-01-03_A10458,False,0.0,0.0
337604,2023-04-17,A10458,1.0,112.0,1796.7,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,Monday,False,2023-04-17,2023-04-17_A10458,False,0.0,0.0


In [64]:
d11[d11['TARGET2'] == 1]#xq sólo están la mitad de los defaults en target??

,DATE,ID_BRANCH,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,TX_MO,TOTAL_AMOUNT_MO,DSO_DEPO_DAY,...,AGENCY,ID_GROUP_COMPANY,START_DATE,day_of_week,filtro,DATE1,DATE_ID_BRANCH,filtro2,TARGET2,TARGET3
337595,2023-04-04,A10458,2.0,184.0,159.01,0.0,0.0,NaN,NaN,1.0,...,NaN,NaN,NaN,Tuesday,False,2023-04-04,2023-04-04_A10458,False,1.0,0.0


In [65]:
d11[d11['TARGET3'] == 1]

,DATE,ID_BRANCH,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,TX_MO,TOTAL_AMOUNT_MO,DSO_DEPO_DAY,...,AGENCY,ID_GROUP_COMPANY,START_DATE,day_of_week,filtro,DATE1,DATE_ID_BRANCH,filtro2,TARGET2,TARGET3
337598,2023-04-08,A10458,2.0,252.0,201.26,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,Saturday,False,2023-04-08,2023-04-08_A10458,False,0.0,1.0


In [48]:
# casteando id_branch como str? NOPPP
d3['ID_BRANCH'] =d3['ID_BRANCH'].astype(str)

/tmp/ipykernel_421/2687558212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3['ID_BRANCH'] =d3['ID_BRANCH'].astype(str)


In [28]:
d3 = d8[d8['DEFAULT'] == 1]

In [29]:
d3['ID_BRANCH'].unique()#tienen un espacio dsp del código 'A00152    '

array(['A00216', 'A00219', 'A00337', ..., 'A26036', 'A26081', 'AB0110'],
      dtype=object)

In [47]:
d3[d3['ID_BRANCH'] == 'A00152    ']

,DATE,ID_BRANCH,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,TX_MO,TOTAL_AMOUNT_MO,DSO_DEPO_DAY,INACTIVE_DAYS,DEFAULT,AGENCY,ID_GROUP_COMPANY,START_DATE,day_of_week,TARGET,filtro,TARGET2
5048,2022-04-11,A00152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,1.0,NaN,NaN,NaN,Monday,0.0,False,0.0
5062,2022-11-24,A00152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600.0,1.0,NaN,NaN,NaN,Thursday,0.0,False,0.0


In [118]:
d3['DEFAULT'].sum()

5226.0

In [46]:
d3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5226 entries, 5048 to 8412464
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                5226 non-null   datetime64[ns]
 1   ID_BRANCH           5226 non-null   object        
 2   TX                  0 non-null      float64       
 3   AMOUNT              0 non-null      float64       
 4   BALANCE             0 non-null      float64       
 5   TX_check            0 non-null      float64       
 6   TOTAL_AMOUNT_check  0 non-null      float64       
 7   TX_MO               0 non-null      float64       
 8   TOTAL_AMOUNT_MO     0 non-null      float64       
 9   DSO_DEPO_DAY        0 non-null      float64       
 10  INACTIVE_DAYS       5226 non-null   float64       
 11  DEFAULT             5226 non-null   float64       
 12  AGENCY              0 non-null      object        
 13  ID_GROUP_COMPANY    0 non-null      object     

In [126]:
d3['ID_BRANCH'] =d3['ID_BRANCH'].astype(str)

/tmp/ipykernel_21/432084206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3['ID_BRANCH'] =d3['ID_BRANCH'].astype(str)


In [128]:
# filtro por default =1 pero al filtrar por esos id_branches la tabla resultante no tiene default....
d11 = d3[d3['ID_BRANCH'] == 'A25927']

In [129]:
d11['DEFAULT'].sum()

0.0

In [130]:
d11

,DATE,ID_BRANCH,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,TX_MO,TOTAL_AMOUNT_MO,DSO_DEPO_DAY,...,AGENCY,ID_GROUP_COMPANY,START_DATE,day_of_week,TARGET,prb1,prb2,prb3,filtro,TARGET2


In [123]:
d12 = df[df['ID_BRANCH'] == 'A25927']
d12

,DATE,ID_BRANCH,TX,AMOUNT,BALANCE,TX_check,TOTAL_AMOUNT_check,TX_MO,TOTAL_AMOUNT_MO,DSO_DEPO_DAY,...,DEFAULT,AGENCY,ID_GROUP_COMPANY,START_DATE,day_of_week,TARGET,prb1,prb2,prb3,filtro
8236826,2023-12-27,A25927,1.0,1485.00,0.0000,0.0,0.0,NaN,NaN,NaN,...,0.0,TX1182,USA017719,2023-12-27,Wednesday,0.0,0.0,0.0,0.0,False
8236827,2024-01-17,A25927,2.0,2075.00,-7.5000,0.0,0.0,NaN,NaN,0.0,...,0.0,TX1182,USA017719,2023-12-27,Wednesday,0.0,0.0,0.0,0.0,False
8236828,2024-01-18,A25927,5.0,4935.59,2092.5000,0.0,0.0,NaN,NaN,0.0,...,0.0,TX1182,USA017719,2023-12-27,Thursday,0.0,0.0,0.0,0.0,False
8236829,2024-01-19,A25927,5.0,5127.00,7697.4959,0.0,0.0,NaN,NaN,0.0,...,0.0,TX1182,USA017719,2023-12-27,Friday,0.0,0.0,0.0,0.0,False
8236830,2024-01-29,A25927,14.0,3020.00,3498.1559,0.0,0.0,NaN,NaN,NaN,...,0.0,TX1182,USA017719,2023-12-27,Monday,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8237590,2024-01-29,A25927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,TX1182,USA017719,2023-12-27,Monday,0.0,0.0,0.0,0.0,True
8237591,2024-01-30,A25927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,TX1182,USA017719,2023-12-27,Tuesday,0.0,0.0,0.0,0.0,True
8237592,2024-01-31,A25927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,TX1182,USA017719,2023-12-27,Wednesday,0.0,0.0,0.0,0.0,True
8237593,2024-02-01,A25927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,TX1182,USA017719,2023-12-27,Thursday,0.0,0.0,0.0,0.0,False


In [22]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4374562 entries, 0 to 4374561
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   DATE                object 
 1   AGENCY              object 
 2   AMOUNT              float64
 3   BALANCE             float64
 4   ID_BRANCH           object 
 5   TOTAL_AMOUNT_MO     float64
 6   TOTAL_AMOUNT_check  float64
 7   TX                  float64
 8   TX_MO               float64
 9   TX_check            float64
dtypes: float64(7), object(3)
memory usage: 333.8+ MB


In [21]:
result_df.describe()

,AMOUNT,BALANCE,TOTAL_AMOUNT_MO,TOTAL_AMOUNT_check,TX,TX_MO,TX_check
count,4.372404e+06,4.372108e+06,2.860898e+06,4.372404e+06,4.372404e+06,2.860898e+06,4.372404e+06
mean,2.949297e+03,3.354048e+03,1.728412e+02,2.077826e+03,6.429362e+00,3.522520e-01,2.619168e+00
std,8.523923e+03,3.512052e+04,1.878780e+03,1.441641e+04,1.701042e+01,3.996904e+00,1.650885e+01
min,0.000000e+00,-2.853621e+07,-2.961390e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.980000e+03,2.196442e+03,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,0.000000e+00
max,6.079065e+05,2.365007e+06,2.795072e+05,2.427941e+06,8.380000e+02,8.030000e+02,1.555000e+03


In [27]:
result_df.head()

,DATE,AGENCY,AMOUNT,BALANCE,ID_BRANCH,TOTAL_AMOUNT_MO,TOTAL_AMOUNT_check,TX,TX_MO,TX_check
0,2023-01-04,AK0001,1550.00,136.46,A20465,NaN,0.0,2.0,NaN,0.0
1,2023-01-05,AK0001,479.17,1511.46,A20465,NaN,0.0,2.0,NaN,0.0
2,2023-01-07,AK0001,3000.00,2010.63,A20465,NaN,0.0,2.0,NaN,0.0
3,2023-01-11,AK0001,150.00,-59.37,A20465,NaN,0.0,1.0,NaN,0.0
4,2023-01-12,AK0001,500.00,100.63,A20465,NaN,0.0,1.0,NaN,0.0


result_df.to_csv('2023_trx_balance_chq_mo.csv', index = False)

# Sumando MO (money orders)

In [ ]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

df3 = pd.read_sql('''SELECT
   CAST(DATE_FORMAT(a.TRANSACTION_DATE, '%Y-%m-%d') AS DATE) AS DATE,
   0 as unique_senders,
   COUNT(DISTINCT a.ID_MONEYORDERS_SALES) AS TX,
   COUNT(DISTINCT CASE WHEN a.status IN ('C', 'A') THEN a.ID_MONEYORDERS_SALES END) AS TX_CANCELLED,
   SUM(a.AMOUNT) AS TOTAL_AMOUNT,
   COUNT(DISTINCT a.ID_MONEYORDERS_SALES) * 0.39 AS GP,
   r.state,
   b.id_chain,
   g.name_chain,
   r.id_location,
   r.name_location,
   r.id_branch,
   r.name_branch,
   h.kpr,
   r.LATITUD,
   r.LONGITUD,
   r.ZIP,
   f.id_agent,
   'MONEY ORDERS' AS TYPE_PRODUCT
FROM
   AwsDataCatalog.viamericas.vcw_moneyorders_sales a
   LEFT JOIN AwsDataCatalog.viamericas.branch b ON b.ID_BRANCH = a.id_branch
   LEFT JOIN AwsDataCatalog.viamericas.branch_agent f ON f.id_branch =b.id_branch
   LEFT JOIN AwsDataCatalog.viamericas.chain g ON g.id_chain =b.id_chain
   LEFT JOIN AwsDataCatalog.viamericas.BUDGET_LOCATIONS_INCLUDED h ON h.id_location =b.id_location
   LEFT JOIN AwsDataCatalog.analytics.location_branch_reference r ON r.id_branch = b.id_branch
WHERE
   NOT (b.ID_MAIN_BRANCH LIKE 'M%')
   AND NOT (a.id_branch LIKE 'T%')
   AND b.id_flag_branch = 'A'
   AND DATE_FORMAT(TRANSACTION_DATE, '%Y-%m') > '2022-12' AND DATE_FORMAT(TRANSACTION_DATE, '%Y-%m') < '2024-02' 
   AND  NOT r.id_location = 'NULL'
   AND b.name_branch not like '%DUMMY%'
 GROUP BY
   CAST(DATE_FORMAT(a.TRANSACTION_DATE, '%Y-%m-%d') AS DATE),
   r.state,
   b.id_chain,
   g.name_chain,
   r.id_location,
   r.name_location,
   r.id_branch,
   r.name_branch,
   h.kpr,
   r.LATITUD,
   r.LONGITUD,
   r.ZIP,
   f.id_agent
  ORDER BY 
   DATE;''', conn)

## Getting default values

In [ ]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

df3 = pd.read_sql('''WITH Payment_Agreeement as(
----------------------------------------------------------------------------------------------
--Agencias que han realizado un acuerdo de pago
----------------------------------------------------------------------------------------------
	SELECT
			DISTINCT
			REPLACE(RTRIM(b.ID_BRANCH),'C','A') AS ID_BRANCH --agencia que realizo el acuerdo
			,g.DATE_CREA_MAIN_BRANCH AS DATE_START -- Cuando empezo el acuerdo
			,g.DEBITO_GROUP AS  NEGOTIATED   --De cuanto fue el acuerdo
			,g.CREDITO_GROUP AS PAID -- Cuanto ha pagado la agencia del acuerdo
			,g.BALANCE_GROUP AS BALANCE --Banlance del acuerdo
	FROM	  AwsDataCatalog.viamericas.GROUP_BRANCH g INNER JOIN
			  AwsDataCatalog.viamericas.BRANCH b ON g.ID_MAIN_BRANCH = b.ID_MAIN_BRANCH
	WHERE	g.DEBITO_GROUP IS NOT NULL
			AND b.ID_LOCATION IS NOT NULL
			AND b.ID_BRANCH LIKE 'C%'
            AND g.DATE_CREA_MAIN_BRANCH >= '2022-01-01'
			AND g.DATE_CREA_MAIN_BRANCH < '2024-02-04'
), Audit_Events as(
	SELECT  
			a.DATE_AUDIT
			, b.ID_BRANCH
			, a.OLD_STATUS
			, a.OLD_FLAG
			, a.OLD_COMMENT
			, a.NEW_STATUS
			, a.NEW_FLAG
			, a.NEW_COMMENT
			, a.LAST_INACTIVATION_TYPE
			, i.DESC_INACTIVATION_TYPE
			, ROW_NUMBER() OVER(Partition By b.ID_BRANCH Order By b.ID_BRANCH, a.DATE_AUDIT ASC)As RowNum
	FROM	   AwsDataCatalog.viamericas  . AUDIT_BRANCH_STATUS   a WITH   INNER JOIN
			   AwsDataCatalog.viamericas  . INACTIVATION_TYPES   i WITH   ON i.ID_INACTIVATION_TYPE = a. LAST_INACTIVATION_TYPE   INNER JOIN
			Envio.  AwsDataCatalog.viamericas.BRANCH AS b WITH   ON a.ID_BRANCH = b.ID_BRANCH INNER JOIN
			 Envio  .   AwsDataCatalog.viamericas  .GROUP_BRANCH g ON b.ID_MAIN_BRANCH = g.ID_MAIN_BRANCH
	WHERE	a. DATE_AUDIT   >= '2022-01-01'
			AND a. DATE_AUDIT   < '2024-02-04'
			AND (RTRIM(b.ID_BRANCH) LIKE 'A%')
			AND b.ID_TYPE_BRANCH = 'R'
            
			--AND b.ID_BRANCH IN('A00152','A00205','A00219','A01626','A01849')
			AND NOT (b.ID_BRANCH LIKE 'T%')
), PAST_DUE_AGENTS AS(
----------------------------------------------------------------------------------------------
--Agencias que han caido en PAST DUE de mas de 15 dias
----------------------------------------------------------------------------------------------
SELECT	
		DISTINCT i.ID_BRANCH
FROM	Audit_Events AS i LEFT JOIN
		Audit_Events AS a ON i.ID_BRANCH = a.ID_BRANCH AND i.DATE_AUDIT < a.DATE_AUDIT AND a.OLD_FLAG = 'I' AND a. NEW_FLAG   = 'A' AND i.RowNum = a.RowNum - 1 LEFT JOIN
		Payment_Agreeement p ON i.ID_BRANCH = p.ID_BRANCH AND CAST(i.DATE_AUDIT AS DATE) >= CAST(p.DATE_START AS DATE)
WHERE	i.OLD_FLAG = 'A'
		AND i. NEW_FLAG   = 'I'
		AND (  date_diff('day',i. DATE_AUDIT  , ISNULL(a.DATE_AUDIT, GETDATE())) >= 15 OR a.DATE_AUDIT IS NULL)
		AND i.LAST_INACTIVATION_TYPE IN(1,3,16)
		--AND i.DESC_INACTIVATION_TYPE IN('PAST DUE / BALANCE VENCIDO','BOUNCED / RETORNO','CHAIN PAST DUE / DEUDA EN CADENA'/*,'BUSINESS CLOSED / CIERRE DEL NEGOCIO'*/)
)
----------------------------------------------------------------------------------------------
--historial de inactivaciones de las agencias que han caido en PAST DUE de mas de 15 dias
----------------------------------------------------------------------------------------------
SELECT	
		i.DATE_AUDIT AS INACTIVATION_DATE
		, i.ID_BRANCH
		,    AwsDataCatalog.viamericas  . sfGetBranchStatus  (i.ID_BRANCH) AS CURRENT_EXTENDED_STATUS -- Status de la agencia a HOY
		, p.DATE_START AS  AGREEMENT_START_DATE   -- FECHA EN LA QUE EMPEZO EL ACUEROD DE PAGO
        ,  date_diff('day', p.DATE_START, i. DATE_AUDIT  ) AS  AGREEMENT_DAYS   -- CUANTO DIAS LLEVA EL ACUERDO A LA FECHA DEL EVENTO DE INACTIVACION DE ESTE REGISTRO
		, ISNULL(p.BALANCE, 0) AS  AGREEMENT_BALANCE   -- BALANCE A HOY DEL ACUERDO
		, ISNULL(p.NEGOTIATED, 0) AS  AGREEMENT_INITIAL_AMOUNT   -- VALOR INICIAL A HOY DEL ACUERDO
		, ISNULL(p.PAID, 0) AS  AGREEMENT_PAID   -- LO QUE SE A PAGADO A HOY DEL ACUERDO
		, i.OLD_STATUS
		, i.OLD_FLAG
		, i.OLD_COMMENT
		, i.NEW_STATUS
		, i.NEW_FLAG
		, i.NEW_COMMENT
		, i.LAST_INACTIVATION_TYPE
		, i.DESC_INACTIVATION_TYPE as DESC_LAST_INACTIVATION_TYPE
		, a.DATE_AUDIT as ACTIVATION_DATE
		,  date_diff('day',i. DATE_AUDIT  , ISNULL(a.DATE_AUDIT, GETDATE())) AS INACTIVE_DAYS
FROM	Audit_Events AS i INNER JOIN
		PAST_DUE_AGENTS pd ON i.ID_BRANCH = pd.ID_BRANCH LEFT JOIN
		Audit_Events AS a ON i.ID_BRANCH = a.ID_BRANCH AND i.DATE_AUDIT < a.DATE_AUDIT AND a.OLD_FLAG = 'I' AND a. NEW_FLAG   = 'A' AND i.RowNum = a.RowNum - 1 LEFT JOIN
		Payment_Agreeement p ON i.ID_BRANCH = p.ID_BRANCH AND CAST(i.DATE_AUDIT AS DATE) >= CAST(p.DATE_START AS DATE)
WHERE	i.OLD_FLAG = 'A'
		AND i. NEW_FLAG   = 'I'
ORDER BY i.ID_BRANCH, i.DATE_AUDIT ASC; ''', conn)

#necesita adaptación para correr en ambiente athena


In [2]:
df2 = pd.read_csv('Datos Juan.csv')

NameError: name 'pd' is not defined

In [10]:
df2['INACTIVATION_DATE'] = pd.to_datetime(df2['INACTIVATION_DATE'])

In [11]:
#cambiar la fecha INACTIVATION_DATE a formato y-m-d y su nombre a DATE, filtrar sólo los inactive days mayores a 15 y llamarlos default
df2['DATE'] = df2['INACTIVATION_DATE'].dt.date

In [12]:
df3 = df2[df2['INACTIVE_DAYS'] > 6] #me quedo con 5 días inactivos en adelante

In [27]:
df3['DEFAULT'] = 1

In [25]:
df3.head()

,DATE,ID_BRANCH,INACTIVE_DAYS
13,2022-04-11,A00152,9
27,2022-11-24,A00152,600
29,2023-06-26,A00216,386
55,2023-08-21,A00219,38
67,2024-01-17,A00219,8


In [28]:
lista =['DATE', 'ID_BRANCH', 'INACTIVE_DAYS','DEFAULT']
df3 = df3[lista]

In [29]:
df3.shape

(5226, 4)

In [16]:
df3['DATE'] = pd.to_datetime(df3['DATE'])

In [18]:
result_df = pd.read_csv('2023_trx_balance_chq_mo.csv')

In [19]:
result_df['DATE'] = pd.to_datetime(result_df['DATE'])

In [30]:
merged_df = pd.merge(result_df, df3, on=['DATE', 'ID_BRANCH'], how='outer')

In [32]:
merged_df['DEFAULT'].unique()

array([ 1., nan])

In [22]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4379788 entries, 0 to 4379787
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DATE                datetime64[ns]
 1   AGENCY              object        
 2   AMOUNT              float64       
 3   BALANCE             float64       
 4   ID_BRANCH           object        
 5   TOTAL_AMOUNT_MO     float64       
 6   TOTAL_AMOUNT_check  float64       
 7   TX                  float64       
 8   TX_MO               float64       
 9   TX_check            float64       
 10  INACTIVE_DAYS       float64       
dtypes: datetime64[ns](1), float64(8), object(2)
memory usage: 367.6+ MB


In [33]:
merged_df.to_csv('base_deafult_ABT.csv', index = False)

## Make looking back variables

In [5]:
df = pd.read_csv('base_deafult_ABT.csv')

In [10]:
df.tail()

,DATE,AGENCY,AMOUNT,BALANCE,ID_BRANCH,TOTAL_AMOUNT_MO,TOTAL_AMOUNT_check,TX,TX_MO,TX_check,INACTIVE_DAYS,DEFAULT,day_of_week,TARGET
4379783,2024-02-03,VA1047,3990.00,0.0000,A26180,0.0,0.00,10.0,0.0,0.0,NaN,0.0,Saturday,0.0
4379784,2024-02-03,OH0244,8180.96,2042.0000,A26182,0.0,0.00,18.0,0.0,0.0,NaN,0.0,Saturday,0.0
4379785,2024-02-03,CA0803,0.00,0.0000,A26184,0.0,0.00,0.0,0.0,0.0,NaN,0.0,Saturday,0.0
4379786,2024-02-03,VA0001,7731.00,4638.6391,AB0106,0.0,18221.58,24.0,0.0,9.0,NaN,0.0,Saturday,0.0
4379787,2024-02-03,VA0003,4224.00,2830.2970,AB0110,0.0,759.76,11.0,0.0,3.0,NaN,0.0,Saturday,0.0


In [7]:
df['DEFAULT'].unique()

array([ 1., nan])

In [8]:
#fillna
values = {"AMOUNT": 0, "BALANCE": 0, "TOTAL_AMOUNT_MO": 0, "TOTAL_AMOUNT_check": 0, 'TX':0, 'TX_MO':0, 'TX_check':0, 'DEFAULT':0}
df.fillna(value=values, inplace=True)

In [9]:
# add day of week
df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()

In [12]:
# Target default mondays inactivation wednesday next week, lag 9 days, at slicing could serve tue and wed also
df['TARGET'] = df.set_index('DATE').groupby('ID_BRANCH', sort=False)['DEFAULT'].shift(-9).to_numpy()
# You can fill this NaN with 0 or another appropriate value
df['TARGET'] = df['TARGET'].fillna(0)

In [13]:
df.to_csv('deafult_ABT1.csv', index = False)

# With target added

In [5]:
df = pd.read_csv('deafult_ABT1.csv')

In [6]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [7]:
def calculate_rolling_means(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).mean().round(2).to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_mean_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df


In [8]:
df = calculate_rolling_means(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], '30D')
df = calculate_rolling_means(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], '7D')

In [9]:
def calculate_rolling_max(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).max().to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_max_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [10]:
df = calculate_rolling_max(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], '7D')
df = calculate_rolling_max(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], '30D')

In [11]:

def calculate_lag(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].shift(window).to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'lag_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [12]:
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 1)

In [13]:
df.columns

Index(['DATE', 'AGENCY', 'AMOUNT', 'BALANCE', 'ID_BRANCH', 'TOTAL_AMOUNT_MO',
       'TOTAL_AMOUNT_check', 'TX', 'TX_MO', 'TX_check', 'INACTIVE_DAYS',
       'DEFAULT', 'day_of_week', 'TARGET', 'roll_mean_30D_AMOUNT',
       'roll_mean_30D_BALANCE', 'roll_mean_30D_TOTAL_AMOUNT_MO',
       'roll_mean_30D_TOTAL_AMOUNT_check', 'roll_mean_30D_TX',
       'roll_mean_30D_TX_MO', 'roll_mean_30D_TX_check',
       'roll_mean_30D_DEFAULT', 'roll_mean_7D_AMOUNT', 'roll_mean_7D_BALANCE',
       'roll_mean_7D_TOTAL_AMOUNT_MO', 'roll_mean_7D_TOTAL_AMOUNT_check',
       'roll_mean_7D_TX', 'roll_mean_7D_TX_MO', 'roll_mean_7D_TX_check',
       'roll_mean_7D_DEFAULT', 'roll_max_7D_AMOUNT', 'roll_max_7D_BALANCE',
       'roll_max_7D_TOTAL_AMOUNT_MO', 'roll_max_7D_TOTAL_AMOUNT_check',
       'roll_max_7D_TX', 'roll_max_7D_TX_MO', 'roll_max_7D_TX_check',
       'roll_max_7D_DEFAULT', 'roll_max_30D_AMOUNT', 'roll_max_30D_BALANCE',
       'roll_max_30D_TOTAL_AMOUNT_MO', 'roll_max_30D_TOTAL_AMOUNT_check',
   

In [14]:
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 2)
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 7)
df = calculate_lag(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 30)

In [15]:
def calculate_rolling_sum(df, columns, window):
    # Create an empty DataFrame to hold the results
    result_df = pd.DataFrame()

    # Loop through each column in the list of columns
    for column in columns:
        # Calculate the rolling mean for the current column
        rolling_mean = df.set_index('DATE').groupby('ID_BRANCH', sort=False)[column].rolling(window).sum().to_numpy()

        # Add the rolling mean to the result DataFrame
        result_df[f'roll_sum_{window}_{column}'] = rolling_mean

    # Concatenate the original DataFrame with the result DataFrame
    result_df = pd.concat([df, result_df], axis=1)

    return result_df

In [16]:
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 7)
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 30)
df = calculate_rolling_sum(df, ["AMOUNT", "BALANCE", "TOTAL_AMOUNT_MO", "TOTAL_AMOUNT_check", 'TX', 'TX_MO', 'TX_check', 'DEFAULT'], 60)

df.to_csv('deafult_ABT2.csv', index = False)

In [5]:
pip install ydata_profiling --upgrade

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install  typing --upgrade

In [7]:
import ydata_profiling

from ydata_profiling import ProfileReport

In [24]:
df = pd.read_csv('deafult_ABT2.csv')

In [27]:
df.tail(30)

,DATE,AGENCY,AMOUNT,BALANCE,ID_BRANCH,TOTAL_AMOUNT_MO,TOTAL_AMOUNT_check,TX,TX_MO,TX_check,...,roll_sum_30_TX_check,roll_sum_30_DEFAULT,roll_sum_60_AMOUNT,roll_sum_60_BALANCE,roll_sum_60_TOTAL_AMOUNT_MO,roll_sum_60_TOTAL_AMOUNT_check,roll_sum_60_TX,roll_sum_60_TX_MO,roll_sum_60_TX_check,roll_sum_60_DEFAULT
4379758,2024-02-03,AZ0220,376.00,953.8101,A26122,0.0,0.00,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379759,2024-02-03,CA4698,5060.00,3636.9980,A26123,0.0,0.00,15.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379760,2024-02-03,TN0578,3153.00,7696.8800,A26124,0.0,0.00,10.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379761,2024-02-03,CA4699,310.00,4135.2900,A26126,0.0,0.00,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379762,2024-02-03,SC0174,367.00,667.0000,A26127,0.0,0.00,4.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379763,2024-02-03,VA1045,2600.00,6243.0000,A26130,0.0,0.00,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379764,2024-02-03,NJ0245,5272.00,9411.0000,A26134,0.0,0.00,16.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379765,2024-02-03,NY1578,1100.00,0.0000,A26135,0.0,0.00,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379766,2024-02-03,IL0916,2589.00,0.0000,A26136,0.0,0.00,9.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4379767,2024-02-03,NY1579,3550.00,2235.0000,A26137,0.0,0.00,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df['TARGET'].sum()

166.0

In [10]:
df['DEFAULT'].sum()

5226.0

In [11]:
# Target default mondays inactivation wednesday next week, lag 9 days, at slicing could serve tue and wed also
df['TARGET'] = df.set_index('DATE').groupby('ID_BRANCH', sort=False)['DEFAULT'].shift(-9).to_numpy()
# You can fill this NaN with 0 or another appropriate value
df['TARGET'] = df['TARGET'].fillna(0)

In [13]:
df.columns

Index(['DATE', 'AGENCY', 'AMOUNT', 'BALANCE', 'ID_BRANCH', 'TOTAL_AMOUNT_MO',
       'TOTAL_AMOUNT_check', 'TX', 'TX_MO', 'TX_check',
       ...
       'roll_sum_30_TX_check', 'roll_sum_30_DEFAULT', 'roll_sum_60_AMOUNT',
       'roll_sum_60_BALANCE', 'roll_sum_60_TOTAL_AMOUNT_MO',
       'roll_sum_60_TOTAL_AMOUNT_check', 'roll_sum_60_TX', 'roll_sum_60_TX_MO',
       'roll_sum_60_TX_check', 'roll_sum_60_DEFAULT'],
      dtype='object', length=102)

In [14]:
#slice de 1 mes
df['DATE'] = pd.to_datetime(df['DATE'])

In [19]:
df1 = df[df['DATE']> '2023-12-31']

In [20]:
df1 = df1.drop(['TARGET'], axis=1)

In [ ]:
profile = ProfileReport(df1, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Exportar el informe en formato HTML
profile.to_file("informe_variables_default_viam_1mes.html")

In [50]:
# Create a new column 'sales_diff' that represents the difference
# in sales from the previous period
df['sales_diff'] = df['sales'].diff()

# The first value in 'sales_diff' will be NaN, since there's no previous value
# You can fill this NaN with 0 or another appropriate value
df['sales_diff'].fillna(0, inplace=True)

KeyError: 'sales'

In [ ]:
 Create new columns 'sales_lag1', 'sales_lag2', ..., 'sales_lagN'
# that represent the sales from the previous 1, 2, ..., N periods
for i in range(1, N+1):
    df[f'sales_lag{i}'] = df['sales'].shift(i)

# The first N values in 'sales_lag1', 'sales_lag2', ..., 'sales_lagN' will be NaN,
# since there's no previous value
# You can fill these NaNs with 0 or another appropriate value
df.fillna(0, inplace=True)

In [ ]:
# Create a new column 'sales_pct_change' that represents the percentage change
# in sales from the previous period
df['sales_pct_change'] = df['sales'].pct_change() #compara entre filas
df.pct_change(axis='columns', periods=-1) #compara entre columnas
#Despite the name of this method, it calculates fractional change (also known as per unit change or relative change) and not percentage change. If you need the percentage change, multiply these values by 100.

In [16]:
merged_df['DATE'] = pd.to_datetime(merged_df['DATE'])
merged_df = merged_df.sort_values(by=['ID_BRANCH','DATE'])
merged_df['roll_mean_month_balance'] = merged_df.set_index('DATE').groupby('ID_BRANCH', sort=False)['BALANCE'].rolling('30D').mean().round(2).to_numpy()

In [19]:
df.head()

,DATE,AGENCY,AMOUNT,BALANCE,ID_BRANCH,TOTAL_AMOUNT_MO,TOTAL_AMOUNT_check,TX,TX_MO,TX_check,...,roll_max_7D_TX_check,roll_max_7D_DEFAULT,roll_max_30D_AMOUNT,roll_max_30D_BALANCE,roll_max_30D_TOTAL_AMOUNT_MO,roll_max_30D_TOTAL_AMOUNT_check,roll_max_30D_TX,roll_max_30D_TX_MO,roll_max_30D_TX_check,roll_max_30D_DEFAULT
0,2022-01-03,NaN,0.0,0.0,A09098,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2022-01-03,NaN,0.0,0.0,A10537,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2022-01-03,NaN,0.0,0.0,A15244,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2022-01-03,NaN,0.0,0.0,A17226,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2022-01-04,NaN,0.0,0.0,A13770,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Si las escalas estan allover the place
from sklearn.preprocessing import StandardScaler

# Assuming you have a DataFrame `df` with columns 'sales' and 'binary_var'

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the 'sales' column and transform it
df['sales_scaled'] = scaler.fit_transform(df[['sales']])

# Now 'sales_scaled' has a mean of 0 and a standard deviation of 1
# 'binary_var' is not affected by the standardization